# PISCES minimize residuals

This is an attempt at shifting the polychromes by small amounts in y to account for small vertical shifts in the image. The idea is to compute the rms of the residuals for a set of multiple polychrome cubes, then update the wavelength calibration by the vertical value that minimizes the residuals.

## Load crispy stuff

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'

import sys,os
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
import logging as log
from crispy.tools.initLogger import getLogger
log = getLogger('main')
from crispy.tools.image import Image
from crispy.PISCESparams import Params
codefolder = '../../../crispy'
par = Params(codefolder)
par.lamlist = np.arange(605.,730.,10)
par.filelist = [par.wavecalDir+'det'+str(wav)+'.fits' for wav in par.lamlist]


## Reduce with existing wavecal

In [2]:
from crispy.IFS import reduceIFSMap
cube = reduceIFSMap(par,par.codeRoot+"/Inputs/BB.fits",method='lstsq')
resid = Image(par.exportDir+"/BB_red_lstsq_resid.fits")


crispy - INFO - Read data from HDU 1 of ../../../crispy/Inputs/BB.fits
crispy - INFO - Reduced cube will have 11 wavelength bins
crispy - INFO - Writing data to ../../../crispy/SimResults/BB_red_lstsq.fits
crispy - INFO - Writing data to ../../../crispy/SimResults/BB_red_lstsq_resid.fits
crispy - INFO - Elapsed time: 3.899058s
crispy - INFO - Read data from HDU 1 of ../../../crispy/SimResults/BB_red_lstsq_resid.fits


In [3]:
rms = []
delta = []
delta += [0.0]
ysize,xsize = Image(filename=par.filelist[0]).data.shape
mask=np.ones((ysize,xsize))    
y = np.arange(ysize)
x = np.arange(xsize)
x -= xsize//2
y -= ysize//2
x, y = np.meshgrid(x, y)

r = np.sqrt(x**2 + y**2)
mask = (r<min(ysize,xsize)//2)

rms += [np.nanstd(resid.data[mask])]
print rms

crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det605.0.fits
[35.921997]


## Now shift the wavecal manually

In [4]:
from crispy.tools.wavecal import buildcalibrations
for dy in np.arange(0.01,0.05,0.02):
    lamsol = np.loadtxt(os.path.join(par.wavecalDir, "lamsol.dat"))
    lam = lamsol[:, 0]
    allcoef = lamsol[:, 1:]
    for i in range(lamsol.shape[0]):
        lamsol[i, 11] += dy
    np.savetxt(par.wavecalDir + "lamsol.dat", lamsol)
    
    buildcalibrations(par,
                        inspect=True,
                        genwavelengthsol=False, # already done before
                        makehiresPSFlets=False, # already done
                        makePolychrome=True, # update the polychrome with the new wavecal!
                        upsample=3,
                        nsubarr=4,
                        lamsol=lamsol, # custom wavecal!
                     )
    cube = reduceIFSMap(par,par.codeRoot+"/Inputs/BB.fits",method='lstsq')
    resid = Image(par.exportDir+"/BB_red_lstsq_resid.fits")
    delta += [dy]
    rms += [np.nanstd(resid.data[mask])]
    print delta,rms

buildcalibrations(par,
                inspect=True,
                genwavelengthsol=False, # already done before
                makehiresPSFlets=False, # already done
                makePolychrome=True, # update the polychrome with the new wavecal!
                upsample=3,
                nsubarr=4
             )

#retrieve old cal


crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/Calibra_170425/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det605.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det605.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det615.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det625.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det635.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det645.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det655.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det665.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det6

In [5]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                inspect=True,
                genwavelengthsol=False, # already done before
                makehiresPSFlets=False, # already done
                makePolychrome=True, # update the polychrome with the new wavecal!
                upsample=3,
                nsubarr=4
             )


crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/Calibra_170425/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det605.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det605.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det615.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det625.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det635.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det645.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det655.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det665.0.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/Calibra_170425/det6